In [39]:
import pandas as pd
import numpy as np
import os
import wfdb
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import json
from mealpy.evolutionary_based.GA import BaseGA
from mealpy.evolutionary_based import GA
from mealpy import FloatVar
from mealpy import IntegerVar
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import time
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
from mealpy.swarm_based import PSO

In [10]:
file_path = Path('C:/Users/vinay/Downloads/mit-bih-arrhythmia-database-1.0.0/mit-bih-arrhythmia-database-1.0.0')

In [11]:
data_files=[]
annot_files=[]
for file in os.listdir(file_path):
    if('.dat' in file):
        data_files.append(file[:-4])
    elif('.atr' in file):
        annot_files.append(file[:-4])

In [12]:
char_to_int = {}
count = 0 

for file in annot_files:
    path_file = os.path.join(file_path, file)
    annotation = wfdb.rdann(path_file, 'atr') 
    
    for symbol in annotation.symbol:
        if symbol not in char_to_int: 
            char_to_int[symbol] = count
            count += 1 

print(char_to_int)

{'+': 0, 'N': 1, 'A': 2, 'V': 3, '~': 4, '|': 5, 'Q': 6, '/': 7, 'f': 8, 'x': 9, 'F': 10, 'j': 11, 'L': 12, 'a': 13, 'J': 14, 'R': 15, '[': 16, '!': 17, ']': 18, 'E': 19, 'S': 20, '"': 21, 'e': 22}


In [13]:
for file in annot_files:
    path_file = os.path.join(file_path, file)
    annotation = wfdb.rdann(path_file, 'atr')
    unique, counts = np.unique(annotation.symbol, return_counts=True)
    print(file, dict(zip(unique, counts)), counts.sum(), len(annotation.sample))

100 {'+': 1, 'A': 33, 'N': 2239, 'V': 1} 2274 2274
101 {'+': 1, 'A': 3, 'N': 1860, 'Q': 2, '|': 4, '~': 4} 1874 1874
102 {'+': 5, '/': 2028, 'N': 99, 'V': 4, 'f': 56} 2192 2192
103 {'+': 1, 'A': 2, 'N': 2082, '~': 6} 2091 2091
104 {'+': 45, '/': 1380, 'N': 163, 'Q': 18, 'V': 2, 'f': 666, '~': 37} 2311 2311
105 {'+': 1, 'N': 2526, 'Q': 5, 'V': 41, '|': 30, '~': 88} 2691 2691
106 {'+': 41, 'N': 1507, 'V': 520, '~': 30} 2098 2098
107 {'+': 1, '/': 2078, 'V': 59, '~': 2} 2140 2140
108 {'+': 1, 'A': 4, 'F': 2, 'N': 1739, 'V': 17, 'j': 1, 'x': 11, '|': 8, '~': 41} 1824 1824
109 {'+': 1, 'F': 2, 'L': 2492, 'V': 38, '~': 2} 2535 2535
111 {'+': 1, 'L': 2123, 'V': 1, '~': 8} 2133 2133
112 {'+': 1, 'A': 2, 'N': 2537, '~': 10} 2550 2550
113 {'+': 1, 'N': 1789, 'a': 6} 1796 1796
114 {'+': 3, 'A': 10, 'F': 4, 'J': 2, 'N': 1820, 'V': 43, '|': 1, '~': 7} 1890 1890
115 {'+': 1, 'N': 1953, '|': 6, '~': 2} 1962 1962
116 {'+': 1, 'A': 1, 'N': 2302, 'V': 109, '~': 8} 2421 2421
117 {'+': 1, 'A': 1, 'N': 153

In [14]:
all_signals = []
all_labels = []

for i in range(48):
    data, field = wfdb.rdsamp(os.path.join(file_path, data_files[i]))
    data = data[:, 0]
    
    annot = wfdb.rdann(os.path.join(file_path, annot_files[i]), 'atr')
    segmented_signals = [data[max(0, peak - 100):min(len(data), peak + 100)] for peak in annot.sample]
    
    segmented_array = np.array([
        np.pad(signal, (0, 200 - len(signal)), mode='edge') if len(signal) < 200 else signal
        for signal in segmented_signals
    ])
    
    labels = annot.symbol[:len(segmented_array)]  

    all_signals.append(segmented_array)
    all_labels.append(labels)

In [35]:
f_X_train, f_X_test, f_y_train, f_y_test = [], [], [], []
for i in range(48):
    X = np.array(all_signals[i]).reshape(-1, 200)
    y = np.array(all_labels[i]).flatten()
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.25,
        random_state=42
    )
    f_X_train.append(X_train)
    f_X_test.append(X_test)
    f_y_train.extend(y_train)  
    f_y_test.extend(y_test)

In [36]:
import numpy as np
from sklearn.utils import resample

# Stack and reshape the data
f_X_train = np.vstack(f_X_train).reshape(-1, 200, 1)
f_X_test = np.vstack(f_X_test).reshape(-1, 200, 1)

# Convert labels to integers
y_train_int = [char_to_int.get(char, -1) for char in f_y_train]
y_test_int = [char_to_int.get(char, -1) for char in f_y_test]
y_train_int = [y for y in y_train_int if y != -1]
y_test_int = [y for y in y_test_int if y != -1]

# One-hot encode the labels
num_classes = len(char_to_int)
f_y_train = to_categorical(y_train_int, num_classes=num_classes)
f_y_test = to_categorical(y_test_int, num_classes=num_classes)

# Print original shapes
print(f"Original Train Shape: {f_X_train.shape}, Test Shape: {f_X_test.shape}")
print(f"Original Train Labels Shape: {f_y_train.shape}, Test Labels Shape: {f_y_test.shape}")

# Reduce dataset size to 1%
def reduce_dataset_size(X, y, fraction=0.01):
    num_samples = int(X.shape[0] * fraction)
    indices = np.random.choice(X.shape[0], num_samples, replace=False)
    return X[indices], y[indices]

# Reduce training and testing data
f_X_train, f_y_train = reduce_dataset_size(f_X_train, f_y_train, fraction=0.15)
f_X_test, f_y_test = reduce_dataset_size(f_X_test, f_y_test, fraction=0.1)

# Print reduced shapes
print(f"Reduced Train Shape: {f_X_train.shape}, Test Shape: {f_X_test.shape}")
print(f"Reduced Train Labels Shape: {f_y_train.shape}, Test Labels Shape: {f_y_test.shape}")

Original Train Shape: (84470, 200, 1), Test Shape: (28177, 200, 1)
Original Train Labels Shape: (84470, 23), Test Labels Shape: (28177, 23)
Reduced Train Shape: (12670, 200, 1), Test Shape: (2817, 200, 1)
Reduced Train Labels Shape: (12670, 23), Test Labels Shape: (2817, 23)


In [17]:
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("history", exist_ok=True)

In [28]:
class HybridModel(tf.keras.Model):
    def __init__(self, config, num_classes):
        super(HybridModel, self).__init__()
        
        # Extract hyperparameters from config
        feature_extractor = config[8]  # CNN (0) or RCNN (1)
        sequence_model = config[9]  # BiLSTM (0) or GRU (1)
        num_cnn_layers = int(config[0])
        num_rnn_layers = int(config[1])
        dropout = config[3]
        initial_filters = 2 ** int(config[4])  # Convert to power of 2
        initial_kernel = int(config[5])  # Initial kernel size
        stride = int(config[6])
        initial_hidden_size = 2 ** int(config[10])  # Convert to power of 2
        fc_neurons = 2 ** int(config[11])  # Fully connected neurons (power of 2)

        # Input Layer
        input_layer = layers.Input(shape=(200, 1))  # Default input size of 200
        x = input_layer

        # CNN Feature Extractor (Filters Increase, Kernel Size Decreases)
        num_filters = initial_filters
        kernel_size = initial_kernel
        for _ in range(num_cnn_layers):
            x = layers.Conv1D(filters=num_filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
            x = layers.Dropout(dropout)(x)

            num_filters = min(256, num_filters * 2)  # Filters Increase (Cap at 256)
            kernel_size = max(3, kernel_size - 1)  # Kernel Size Decreases (Min 3)

        # Handle RCNN (CNN + RNN)
        if feature_extractor == 1:
            x = layers.Reshape((-1, num_filters // 2))(x)  # Flatten for RNN
            hidden_size = initial_hidden_size
            for _ in range(num_rnn_layers):
                if sequence_model == 0:
                    x = layers.Bidirectional(layers.LSTM(hidden_size, return_sequences=True))(x)
                else:
                    x = layers.Bidirectional(layers.GRU(hidden_size, return_sequences=True))(x)
                x = layers.Dropout(dropout)(x)
                hidden_size = max(16, hidden_size // 2)  # Hidden Size Decreases (Min 16)

        # Sequence Model (BiLSTM or GRU) (Hidden Size Decreases)
        hidden_size = initial_hidden_size
        for _ in range(num_rnn_layers):
            if sequence_model == 0:
                x = layers.Bidirectional(layers.LSTM(hidden_size, return_sequences=True))(x)
            else:
                x = layers.Bidirectional(layers.GRU(hidden_size, return_sequences=True))(x)
            x = layers.Dropout(dropout)(x)
            hidden_size = max(16, hidden_size // 2)  # Hidden Size Decreases

        # Global Pooling & Fully Connected Layer
        x = layers.GlobalAveragePooling1D()(x)
        x = layers.Dense(fc_neurons, activation='relu')(x)
        output_layer = layers.Dense(num_classes, activation='softmax')(x)  # Multi-class classification
        
        self.model = models.Model(inputs=input_layer, outputs=output_layer)

    def call(self, inputs):
        return self.model(inputs)


In [29]:
def format_filename(config):
    """ Convert model config into a valid filename """
    formatted_config = "_".join([f"{x:.4f}" if isinstance(x, float) else str(x) for x in config])
    formatted_config = re.sub(r'[^\w\-_]', '', formatted_config)  # Remove special characters
    return formatted_config

In [30]:
def convert_to_serializable(obj):
    """ Converts NumPy arrays and other non-serializable objects to lists for JSON serialization """
    if isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert NumPy array to a list
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]  # Recursively process lists
    return obj  # Return normal values unchanged

In [37]:
# Assuming you have already preprocessed and prepared your dataset:
# f_X_train, f_X_test, f_y_train, f_y_test

def measure_latency(model, X_test, batch_size=64):
    """ Measure the time it takes for the model to perform inference on the test data """
    start_time = time.time()
    model.predict(X_test, batch_size=batch_size)
    end_time = time.time()
    return end_time - start_time

def train_and_evaluate(config):
    """
    Train and evaluate the model using real ECG data.
    """
    # TensorFlow does not need to worry about device assignments (CUDA is abstracted)
    # Assuming f_X_train and f_X_test are pre-processed numpy arrays
    X_train, X_test = f_X_train, f_X_test
    y_train, y_test = f_y_train, f_y_test
    
    # Use batch size from config (adjusting it to power of 2 for optimization)
    batch_size = 2**(int(config[7])) if len(config) > 7 else 64
    num_classes = y_train.shape[1]  # Number of classes

    # Initialize model
    model = HybridModel(config, num_classes)  # Use the HybridModel class
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config[2]), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    # Train model
    num_epochs = 10
    history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test, y_test))

    # Evaluate on test set
    accuracy = accuracy_score(y_test.argmax(axis=1), model.predict(X_test).argmax(axis=1))

    # Measure real inference latency
    latency = measure_latency(model, X_test, batch_size)

    # Define fitness score (maximize accuracy, minimize latency)
    fitness = accuracy - (latency / 100)

    # Save model
    model_name = f"model_{format_filename(config)}.h5"
    model.save(f"models/{model_name}")

    # Save results
    serializable_config = convert_to_serializable(config)

    result = {
        "config": serializable_config,
        "accuracy": accuracy,
        "latency": latency,
        "fitness": fitness
    }
    result_filename = f"results/{model_name}.json"
    with open(result_filename, "w") as f:
        json.dump(result, f, indent=4)

    print(f"✅ Config: {serializable_config} -> Accuracy: {accuracy:.4f}, Latency: {latency:.2f}ms, Fitness: {fitness:.4f} | Model Saved: {model_name}")
    return fitness

In [32]:
bounds = [
    IntegerVar(lb=1, ub=5),         # 0: Number of CNN layers (Integer)
    IntegerVar(lb=1, ub=5),         # 1: Number of RNN layers (Integer)
    FloatVar(lb=0.0001, ub=0.01),   # 2: Learning rate (Float)
    FloatVar(lb=0.1, ub=0.5),       # 3: Dropout rate (Float)
    IntegerVar(lb=5, ub=8),         # 4: Number of filters in CNN (Integer)
    IntegerVar(lb=3, ub=6),         # 5: Kernel size for CNN (Integer)
    IntegerVar(lb=1, ub=3),         # 6: Stride for CNN (Integer)
    IntegerVar(lb=4, ub=7),         # 7: Batch size (Integer)
    IntegerVar(lb=0, ub=1),         # 8: CNN/RCNN (Integer: 0 = CNN, 1 = RCNN)
    IntegerVar(lb=0, ub=1),         # 9: GRU/BiLSTM (Integer: 0 = BiLSTM, 1 = GRU)
    IntegerVar(lb=6, ub=9),         # 10: Hidden size for RNN (Integer)
    IntegerVar(lb=5, ub=8),         # 11: Fully connected layer neurons (Integer)
]

In [ ]:
# Define problem dictionary for Mealpy
problem_dict = {
    "obj_func": train_and_evaluate,
    "bounds": bounds,
    "minmax": "max"  # We want to maximize the fitness function
}

# Run Genetic Algorithm optimization
print("\nRunning Genetic Algorithm for Model Optimization...\n")
ga_model = GA.BaseGA(epoch=10, pop_size=5, pc=0.9, pm=0.05)  # 10 generations, 5 models per generation
best_solution = ga_model.solve(problem_dict)

# Extract best architecture and fitness score
best_architecture = best_solution.solution
best_fitness = best_solution.target.fitness

# Save best model details
best_model_info = {
    "best_architecture": best_architecture,
    "best_fitness": best_fitness
}
with open("results/best_model.json", "w") as f:
    json.dump(best_model_info, f, indent=4)

# Print best model
print(f"\n🏆 Best Model Configuration: {best_architecture}, Fitness Score: {best_fitness}")


Running Genetic Algorithm for Model Optimization...



ValueError: Please set at least one stopping condition with parameter 'max_epoch' or 'max_fe' or 'max_time' or 'max_early_stop'